In [1]:
# import lucem_illud #pip install -U git+git://github.com/Computational-Content-Analysis-2018/lucem_illud.git
import gensim
import requests
import nltk #For stop words and stemmers
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import sklearn.metrics.pairwise
import sklearn.manifold
import sklearn.decomposition

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 


import os #For looking through files
import os.path #For managing file paths

import article_analysis.parse as aap
import enchant

%load_ext autoreload
%autoreload 1
%aimport article_analysis.parse
%matplotlib inline

In [2]:
df = pd.read_json('json_amj.txt', lines=True)
df.shape

(5694, 13)

In [3]:
df.head()

,authors,data,discipline,doi,journaltitle,number,pagecount,pagerange,publication-date,reviewed-work,title,type,volume
0,John B. Miner,{'ocr': ['Communications Exodus-The study also...,"[Business & Economics Collection, Management &...",10.2307/254807,The Academy of Management Journal,1,2,127-128,1972-03-01T00:00:00Z,NaN,Success in Management Consulting and the Conce...,research-article,15
1,Charles R. McClure,{'ocr': ['The author's own recommendations are...,"[Business & Economics Collection, Management &...",10.2307/257446,The Academy of Management Review,2,3,312-314,1980-04-01T00:00:00Z,<product><source>\n <source>Information Strate...,,book-review,5
2,NaN,{'ocr': ['The Academy of Management Learning t...,"[Business & Economics Collection, Management &...",10.2307/3069361,The Academy of Management Journal,3,8,449-456,2001-06-01T00:00:00Z,NaN,Front Matter,misc,44
3,Mark S. Mizruchi,{'ocr': ['Acta Sociologica (1994) 37:329-343 S...,"[Sociology, Social Sciences]",10.2307/4200918,Acta Sociologica,4,15,329-343,1994-01-01T00:00:00Z,NaN,Social Network Analysis: Recent Achievements a...,research-article,37
4,"[Masoud Yasai-Ardekani, Peter H. Grinyer]",{'ocr': ['? Academy of Management Journal 1981...,"[Business & Economics Collection, Management &...",10.2307/255569,The Academy of Management Journal,3,16,471-486,1981-09-01T00:00:00Z,NaN,"Strategy, Structure, Size and Bureaucracy",research-article,24


In [4]:
# df['doi'].to_csv('/Users/belikov/data/jstor/doi.csv.gz', compression='gzip')

In [95]:
sample.keys()

dict_keys(['ocr'])

In [96]:
# sample = df.loc[200, 'data']
corpus = dict(zip(df['doi'], [x['ocr'] for x in df['data']]))

In [97]:
len(corpus)

5694

In [ ]:
corpus_ngrams = {}
for j, item in list(zip(range(len(corpus)), corpus.items())):
    doi, article = item
    print('{0} '.format(doi), end='')
    article_phrases = aap.transform_article(article)
    print('{1} '.format(doi, len(article_phrases)), end='')
    ngram_dict = aap.compute_ngrams(article_phrases)
    if j%20 == 0:
        print('\n{0} items, {1:.2f} % complete'.format(j+1, 100*(j+1)/len(corpus)))
    corpus_ngrams[doi] = ngram_dict

10.2307/254807 33 
1 items, 0.02 % complete
10.2307/257446 109 10.2307/3069361 169 10.2307/4200918 801 10.2307/255569 362 10.2307/255822 601 10.2307/255179 146 10.2307/255573 352 10.2307/30040729 817 10.2307/3069433 492 10.2307/257035 518 10.2307/258720 891 10.2307/20159695 805 10.2307/20109897 47 10.2307/255376 275 10.2307/20159163 1166 10.2307/258237 473 10.2307/257943 204 10.2307/255361 274 10.2307/40220141 574 10.2307/255748 195 
21 items, 0.37 % complete
10.2307/257249 362 10.2307/40878158 925 10.2307/258977 806 10.2307/257027 437 10.2307/2667086 744 10.2307/255761 194 10.2307/256212 158 10.2307/255366 80 10.2307/256338 627 10.2307/259103 937 10.2307/256091 185 10.2307/3069368 501 10.2307/2780904 502 10.2307/1556415 457 10.2307/256058 583 10.2307/256557 241 10.2307/20159377 967 10.2307/30040677 472 10.2307/254928 194 10.2307/255645 345 
41 items, 0.72 % complete
10.2307/3069340 632 10.2307/3556659 1414 10.2307/257571 139 10.2307/257616 191 10.2307/254901 41 10.2307/2667048 341 10.

In [152]:
doi_ = '10.2307/3069361'
# doi_ = '10.2307/254807'
doi_ = '10.2307/2667048'
mask = (df['doi'] == doi_)
article = df.loc[mask, 'data'].values[0]['ocr']
# ar_phrases = aap.transform_article(article, verbose=True)
ar_phrases = aap.transform_article(article)
ng_dict = aap.compute_ngrams(ar_phrases)
ng_dict[2]

Counter({('publications', 'received'): 1,
         ('received', 'selected'): 1,
         ('selected', 'books'): 1,
         ('books', 'list'): 1,
         ('list', 'reviewed'): 1,
         ('reviewed', 'future'): 1,
         ('future', 'issues'): 1,
         ('acs', 'zoltan'): 1,
         ('zoltan', 'j'): 1,
         ('j', 'bo'): 1,
         ('bo', 'carlson'): 1,
         ('carlson', 'charlie'): 1,
         ('charlie', 'karlsson'): 1,
         ('karlsson', 'eds'): 1,
         ('entrepreneurship', 'small'): 1,
         ('small', 'medium'): 1,
         ('medium', 'enterprises'): 1,
         ('enterprises', 'macro'): 1,
         ('new', 'york'): 10,
         ('york', 'cambridge'): 2,
         ('cambridge', 'university'): 2,
         ('university', 'press'): 10,
         ('press', 'pp'): 16,
         ('bazerman', 'max'): 1,
         ('judgment', 'managerial'): 1,
         ('managerial', 'decision'): 1,
         ('decision', 'making'): 1,
         ('making', '4th'): 1,
         ('4th', 'ed'